In [4]:
import os
import pickle

In [ ]:
!pip install git+https://github.com/IINemo/isanlp.git

In [6]:
from isanlp import PipelineCommon
from isanlp.processor_remote import ProcessorRemote
from isanlp.ru.processor_mystem import ProcessorMystem
from isanlp.ru.converter_mystem_to_ud import ConverterMystemToUd

In [ ]:
def extr_pairs(tree, text):
    "extract pairs from a tree"
    pp = []
    if tree.left:
        pp.append([text[tree.left.start:tree.left.end],
                   text[tree.right.start:tree.right.end], 
                   tree.relation, tree.nuclearity])
        pp += extr_pairs(tree.left, text)
        pp += extr_pairs(tree.right, text)
    return pp

## isanlp pipelene

In [ ]:
ppl = PipelineCommon([
    (ProcessorRemote('localhost', 3334, '0'),
     ['text'],
     {'sentences': 'sentences',
      'tokens': 'tokens',
      'lemma': 'lemma',
      'syntax_dep_tree': 'syntax_dep_tree',
      'postag': 'ud_postag'}),
    (ProcessorMystem(delay_init=False),
 ['tokens', 'sentences'],
     {'postag': 'postag'}),
    (ConverterMystemToUd(),
     ['postag'],
     {'morph': 'morph',
      'postag': 'postag'}),
    (ProcessorRemote('localhost', 3335, 'default'),
     ['text', 'tokens', 'sentences', 'postag', 'morph', 'lemma', 'syntax_dep_tree'],
     {'rst': 'rst'})
])

## corpus

In [ ]:
texts = []
names = os.listdir('../rus-human-tr')
for name in names:
    if name != 'segmented':
        with open('../rus-human-tr' + '/' + name, 'r', encoding='utf-8') as f:
            text = f.read()
            text = text.split('\n')
            text = ' '.join(text)
            texts.append(text)

In [ ]:
texts[0]

## segmentation and annotation

In [ ]:
%%time

for i in range(len(texts)):
    text = texts[i]
    name = names[i]
    result = ppl(text)
    tree = result['rst'][0]
    tree.to_rs3('file.rs3')
    text = result['text']
    edus = extr_pairs(tree, text)
    print(str(i)) #+ '\n')
    if False:
        with open('../русский авторазметчик эде' + '/' + 'ar_' + name.split('.')[0] + '.pickle', 'wb') as f:
            pickle.dump(edus, f)
        with open('../русский авторазметчик эде' + '/' + 'ar_tree_' + name.split('.')[0] + '.pickle', 'wb') as f:
            pickle.dump(tree, f)    
    with open('../русский авторазметчик эде' + '/' + 'ar_result_' + name.split('.')[0] + '.pickle', 'wb') as f:
        pickle.dump(result, f)